In [29]:
data_location='/content/combinedNetflixData.txt'

In [ ]:
data_location='/content/combinedNetflixData.txt'

In [30]:
# Importing the packages
import pandas as pd
import numpy as np
import seaborn as sns

In [31]:
df=pd.read_csv(data_location,names=['custId','Rating'],usecols=[0,1])

In [32]:
df.head()

,custId,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0


In [33]:
#As we can see here the custId,MovieId(1:,2:,3:,...etc) and Rating is merged togeather in same dataframe so this is ambiguity
#So lets seperate each of the coulmns

In [34]:
#Let's have total movie count
total_movie_count=df.isnull().sum()['Rating']
print(total_movie_count)

4499


In [35]:
#Let's have total customer count
total_customer=df['custId'].nunique()-total_movie_count
print(total_customer)

470758


In [36]:
#Let's have total number of ratings
total_rating=df['custId'].count()-total_customer
print(total_rating)

23587505


In [55]:
#Now let's fix the coulmns
df_copy=df.copy()
movie_curr=None
moive_id=[]
for x in df['custId']:
  if ':' in x:
    movie_curr=int(x.replace(':',''))
  moive_id.append(movie_curr)


In [56]:
df['MovieId']=moive_id

In [57]:
df.head()

,custId,Rating,MovieId
0,1:,NaN,1
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1


In [58]:
#Let's remove the unwanted data
df=df[df['Rating'].notna()]

In [59]:
#Now the data is perfect
df.head()

,custId,Rating,MovieId
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1


In [67]:
from re import M
# We can't remove duplicates here because same user would have rated for many movies.So let's set some threshold
#And then remove the movie which are least rated and also the user which aren't active
movie_list=df.groupby('MovieId')['Rating'].agg(['count'])
movie_list.quantile(0.7)
benchmark_movie=round(movie_list['count'].quantile(0.7),0)
drop_movie_list=movie_list[movie_list['count']<benchmark_movie]
drop_movie_list

,count
MovieId,
1,547
2,145
4,142
5,1140
6,1019
...,...
4494,130
4495,614
4497,714


In [68]:
cust_list=df.groupby('custId')['Rating'].agg(['count'])
benchmark_cust=round(cust_list['count'].quantile(0.7),0)
drop_cust_list=cust_list[cust_list['count']<benchmark_cust]

In [64]:
len(drop_cust_list)
len(drop_movie_list)

3149

In [69]:
df=df[~df['MovieId'].isin(drop_movie_list)]
df=df[~df['custId'].isin(drop_cust_list)]

In [70]:
df.shape[0]

24053764

In [ ]:
# WORKING ON RECOMMENDATION

In [75]:
movie_df=pd.read_csv('/content/NetflixMovieData.csv',names=['MovieId','Year','Name'],usecols=[0,1,2])

In [76]:
movie_df.head()

,MovieId,Year,Name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW


In [78]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357265 sha256=8f99e4a29dea0b2730bfdeeae5d084ab5dbe06be7bb958f7ee1fe1d682a56bf1
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [80]:
from surprise import Dataset,Reader,SVD
from surprise.model_selection import cross_validate

In [81]:
reader=Reader()

In [83]:
data=Dataset.load_from_df(df[['custId','Rating','MovieId']][:100000],reader)

In [ ]:
#Model Building

In [84]:
model=SVD()

In [86]:
cross_validate(model,data,measures=['RMSE'],cv=4)

{'test_rmse': array([18.28543702, 18.28509019, 18.30127712, 18.23096815]),
 'fit_time': (1.4909207820892334,
  1.5226349830627441,
  1.5636980533599854,
  2.09486985206604),
 'test_time': (0.2012937068939209,
  0.14694690704345703,
  0.16221380233764648,
  0.20133686065673828)}

In [ ]:
#Creating filter for recommendation

In [95]:
data_1331154=df[(df['custId']=='1331154') & (df['Rating']==5.0)]

In [100]:
df.head()

,custId,Rating,MovieId
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1


In [96]:
data_1331154

,custId,Rating,MovieId
458308,1331154,5.0,143
1184450,1331154,5.0,270
1991774,1331154,5.0,361
2369367,1331154,5.0,457
2600328,1331154,5.0,482
3417458,1331154,5.0,658
4029215,1331154,5.0,763
5646194,1331154,5.0,1144
7075510,1331154,5.0,1425
7423467,1331154,5.0,1476


In [97]:
list_of_infinite_possibility=movie_df.copy()

In [98]:
list_of_infinite_possibility.reset_index(inplace=True)

In [ ]:
#These are the movies which can possible be recommended

In [99]:
list_of_infinite_possibility=list_of_infinite_possibility[~list_of_infinite_possibility.isin(drop_movie_list)]

In [ ]:
#RECOMMENDATION SYSTEM

In [103]:
list_of_infinite_possibility['Estimated_score']=list_of_infinite_possibility['MovieId'].apply(lambda x:model.predict('44937',x).est)

In [104]:
list_of_infinite_possibility

,index,MovieId,Year,Name,Estimated_score
0,0,1,2003.0,Dinosaur Planet,5
1,1,2,2004.0,Isle of Man TT 2004 Review,5
2,2,3,1997.0,Character,5
3,3,4,1994.0,Paula Abdul's Get Up & Dance,5
4,4,5,2004.0,The Rise and Fall of ECW,5
...,...,...,...,...,...
17764,17764,17766,2002.0,Where the Wild Things Are and Other Maurice Se...,5
17765,17765,17767,2004.0,Fidel Castro: American Experience,5
17766,17766,17768,2000.0,Epoch,5
17767,17767,17769,2003.0,The Company,5
